## import

In [9]:
## ls_env 가상환경에서 실행

import os
import time                              ## 작업 시 소요시간을 체크하기 위함 (본 구문에 start, end 로 계산)
import pandas as pd

path1= "./TAS-wrong"                           ## 파일들이 담겨있는 folder 주소
list_1 = os.listdir(path1)               ## 해당 폴더의 파일목록을 list_1에 입력

In [3]:
## 혹시 파일이 폴더별로 구분되면 좀 귀찮아지기는 할 것임. 그때는 HIE 소스 활용

In [4]:
## 아래 분석 case 별로 양식이 다르기 때문에 별도로 내용을 추출하기 위한 구문이 필요
## 기본적으로 뽑아내야 하는 열의 제목을 txt에서 위치를 확인한 뒤 거기부터 몇줄 뒤 몇번째 칸 이런 형태를 뽑아내게 되어 있음

## 종합분석 case (T, P)
* T code: 종합분석
* P code: 전력실험실 전용분석

In [2]:
def tas_t(line):
    test=[]
    for l in range(580,len(line)):
        if "담당자<" in line[l]:
            test.append(line[l+2][1:-1])
        if "의뢰번호<" in line[l]:
            test.append(line[l+1][20:30]) 
        if "성명<" in line[l]:
            test.append(line[l+1][32:-6])             ## 사람 이름이 세글자가 아닌경우 고려
        if "제목<" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "접수일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "처리일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "STATUS<" in line[l]:
            test.append(line[l+1][32:-6])            ### 진행중인 경우 STATUS 안나옴!!!
        if "의뢰목적<" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰유형<" in line[l]:            
            test.append(line[l+8][9:-1])
        if "제조사<" in line[l]:
            test.append(line[l+1][22:-1])
        if "Comments<" in line[l]:                       ### 여러줄로 되어 있는 comments 임 (</td> 가 나올때까지 여러줄을 모두 comment에 저장)
            comments=""
            for j in range(20):
                if "</TD>" in line[l+j+1]:
                    break
                else:
                    if j==0:
                        comments=(line[l+1][22:-1])                    
                    else:
                        comments=comments+"<br>"+line[l+j+1][:-1]
            test.append(comments)
        if "KeyWord<" in line[l]:
            test.append(line[l+1][32:-6])
        if "분석조건<" in line[l]:
            test.append(line[l+1][32:-6])                     ## 두번째 값을 취해야 함.
        if "분석의견<" in line[l]:
            test.append(line[l+1][33:-6])                                ## 세번째 값을 취해야 함.
        if "분석장비<" in line[l]:
            test.append(line[l+1][32:-6])
    test_T=pd.DataFrame(test).T
    test_T.columns=['의뢰번호','의뢰일','담당자','접수일','처리일',
'제목','의뢰목적','의뢰유형','제조사','Comments',
'의뢰자','처리일2','STATUS','분석조건','분석의견',
'분석장비','KeyWord']
    return test_T

## 특정설비분석 case (E, S)
* E code: 보통
* S code: 지급
* 제조사 공란

In [3]:
def tas_e(line):
    test=[]
    for l in range(600,len(line)):
        if "담당자</" in line[l]:
            test.append(line[l+2][1:-1])
        if "의뢰번호</" in line[l]:
            test.append(line[l+1][20:30]) 
        if "성명</" in line[l]:
            test.append(line[l+1][32:-6])             ## 사람 이름이 세글자가 아닌경우 고려
        if "제목</" in line[l]:
            test.append(line[l+1][33:-6])
        if "의뢰일</" in line[l]:
            test.append(line[l+1][20:-6])
        if "접수일</" in line[l]:
            test.append(line[l+1][20:-6])
        if "처리일</" in line[l]:
            test.append(line[l+1][32:-6])            ### 두번째 값을 취해야 함 (650, 970)
        if "STATUS</" in line[l]:
            test.append(line[l+1][32:-6])            ### 진행중인 경우 STATUS 안나옴!!!
        if "의뢰목적</" in line[l]:
            test.append(line[l+1][32:-7])
        if "의뢰유형</" in line[l]:
            test.append(line[l+8][9:-1])
#    if "제조사" in line[l]:
#        test.append(line[l+1][22:-1])
        if "Comments</" in line[l]:
            test.append(line[l+2][2:-1])
        if "KeyWord</" in line[l]:
            test.append(line[l+1][32:-6])
        if "분석조건</" in line[l]:
            test.append(line[l+1][32:-6])                     ## 두번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석의견</" in line[l]:
            test.append(line[l+1][33:-6])                     ## 세번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석장비</" in line[l]:
            test.append(line[l+1][32:-6])          ### 두번째 값을 취해야 함 (702, 984)
#test.append(line[970][32:-6])                 ## 읽어들여야하는 행번호를 지정 시 중간에 라인 늘어나는 경우 오류 발생
    test_E=pd.DataFrame(test).T
    test_E.columns=['의뢰번호','의뢰일','담당자','처리일','접수일2','제목',
'의뢰목적','의뢰유형','분석장비2','Comments','의뢰자','접수일',
'STATUS','분석조건','분석의견','분석장비','KeyWord']
## 최종 취합 후에 접수일2, 분석장비 2 삭제 필요
    return test_E

## 환경유해물질분석 case (I, V)
* 제조사, 키워드 공란 (14)

In [4]:
def tas_i(line):
    test=[]
    for l in range(600,len(line)):
        if "담당자<" in line[l]:
            test.append(line[l+2][1:-1])
        if "의뢰번호<" in line[l]:
            test.append(line[l+1][20:30]) 
        if "성명<" in line[l]:
            test.append(line[l+1][32:-6])             ## 사람 이름이 세글자가 아닌경우 고려
        if "제목<" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "접수일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "처리일<" in line[l]:
            test.append(line[l+1][32:-6])            ### 두번째 값을 취해야 함 (650, 970)
        if "STATUS<" in line[l]:
            test.append(line[l+1][32:-6])            ### 진행중인 경우 STATUS 안나옴!!!
        if "의뢰목적<" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰유형<" in line[l]:
            test.append(line[l+8][9:-1])
#    if "제조사" in line[l]:
#        test.append(line[l+1][22:-1])
        if "요구사항<" in line[l]:
            test.append(line[l+1][32:-1])
#    if "KeyWord" in line[l]:
#        test.append(line[l+1][32:-6])
        if "분석조건<" in line[l]:
            test.append(line[l+2][2:-1])                     ## 두번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석의견<" in line[l]:
            test.append(line[l+2][2:-1])                     ## 세번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석장비<" in line[l]:
            test.append(line[l+1][32:-6])          ### 두번째 값을 취해야 함 (702, 984)
    test_I=pd.DataFrame(test).T
    test_I.columns=['의뢰번호','의뢰일','담당자','접수일','처리일2','제목',
'의뢰목적','의뢰유형','Comments','의뢰자','처리일',
'STATUS','분석조건','분석의견','분석장비']
## 최종 취합 후에 접수일2, 분석장비 2 삭제 필요
    return test_I

## 환경유해물질분석 case (O)
* 제조사, 키워드 공란 (14)
* 의뢰유형 체크 확인 요망 (2010년 이전과 이후 양식 달라졌는지??)

In [5]:
def tas_o(line):
    test=[]
    for l in range(600,len(line)):
        if "담당자<" in line[l]:
            test.append(line[l+2][2:-1])
        if "의뢰번호<" in line[l]:
            test.append(line[l+1][20:30]) 
        if "성명<" in line[l]:
            test.append(line[l+1][32:-6])             ## 사람 이름이 세글자가 아닌경우 고려
        if "제목<" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "접수일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "처리일<" in line[l]:
            test.append(line[l+1][32:-6])            ### 두번째 값을 취해야 함 (650, 970)
        if "STATUS<" in line[l]:
            test.append(line[l+1][32:-1])            ### 진행중인 경우 STATUS 안나옴!!!
        if "의뢰목적<" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰유형<" in line[l]:
            test.append(line[l+2][32:-1])
#    if "제조사" in line[l]:
#        test.append(line[l+1][22:-1])
        if "요구사항<" in line[l]:
            test.append(line[l+1][32:-6])
#    if "KeyWord" in line[l]:
#        test.append(line[l+1][32:-6])
        if "분석조건<" in line[l]:
            test.append(line[l+1][32:-6])                     ## 두번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석의견<" in line[l]:
            test.append(line[l+1][33:-6])                     ## 세번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석장비<" in line[l]:
            test.append(line[l+1][32:-6])          ### 두번째 값을 취해야 함 (702, 984)
    test_O=pd.DataFrame(test).T
    test_O.columns=['의뢰번호','의뢰일','담당자','접수일','처리일2','제목',
'의뢰목적','의뢰유형','의뢰유형2','Comments','의뢰자','처리일',
'STATUS','분석조건','분석의견','분석장비']
## 최종 취합 후에 접수일2, 분석장비 2 삭제 필요
    return test_O

## 외부분석의뢰(G&P) case (G)
* 제조사 공란 (15)
* 의뢰유형 체크 확인 요망 (샘플에는 빠져있었음)

In [6]:
def tas_g(line):
    test=[]
    for l in range(580,len(line)):
        if "담당자<" in line[l]:
            test.append(line[l+2][1:-1])
        if "의뢰번호<" in line[l]:
            test.append(line[l+1][20:30]) 
        if "성명<" in line[l]:
            test.append(line[l+1][32:-6])             ## 사람 이름이 세글자가 아닌경우 고려
        if "제목<" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "접수일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "처리일<" in line[l]:
            test.append(line[l+1][32:-6])            ### 두번째 값을 취해야 함 (650, 970)
        if "STATUS<" in line[l]:
            test.append(line[l+1][32:-6])            ### 진행중인 경우 STATUS 안나옴!!!
        if "의뢰목적<" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰유형<" in line[l]:
            test.append(line[l+2][32:-1])
#    if "제조사" in line[l]:
#        test.append(line[l+1][22:-1])
        if "Comments<" in line[l]:
            test.append(line[l+1][32:-6])
        if "KeyWord<" in line[l]:
            test.append(line[l+1][32:-6])
        if "분석조건<" in line[l]:
            test.append(line[l+1][32:-6])                     ## 두번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석의견<" in line[l]:
            test.append(line[l+1][33:-6])                     ## 세번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석장비<" in line[l]:
            test.append(line[l+1][32:-6])          ### 두번째 값을 취해야 함 (702, 984)
    test_G=pd.DataFrame(test).T
    test_G.columns=['의뢰번호','의뢰일','담당자','접수일','처리일2','제목',
'의뢰목적','의뢰유형','분석장비2','Comments','의뢰자','처리일',
'STATUS','분석조건','분석의견','분석장비','KeyWord']
## 최종 취합 후에 접수일2, 분석장비 2 삭제 필요
    return test_G

## 클레임분석 case (C)
* 제조사 공란 (15)
* 의뢰유형은 일괄 클레임대응
* 클레임 분석 선택 후 종합분석을 선택하면 제조사를 입력하는 경우가 발생

In [7]:
def tas_c(line):
    test=[]
    for l in range(580,len(line)):
        if "담당자<" in line[l]:
            test.append(line[l+2][1:-1])
        if "의뢰번호<" in line[l]:
            test.append(line[l+1][20:30]) 
        if "성명<" in line[l]:
            test.append(line[l+1][32:-6])             ## 사람 이름이 세글자가 아닌경우 고려
        if "제목<" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "접수일<" in line[l]:
            test.append(line[l+1][20:-6])
        if "처리일<" in line[l]:
            test.append(line[l+1][32:-6])            ### 두번째 값을 취해야 함 (650, 970)
        if "STATUS<" in line[l]:
            test.append(line[l+1][32:-6])            ### 진행중인 경우 STATUS 안나옴!!!
        if "의뢰목적<" in line[l]:
            test.append(line[l+1][32:-6])
#    if "의뢰유형" in line[l]:
#        test.append(line[l+8][9:-1])
#    if "제조사" in line[l]:
#        test.append(line[l+1][22:-1])
        if "이력<" in line[l]:                           ### 클레임분석요청임에도 종합분석/클레임분석을 구분하여 의뢰하게 되며,
            for k in range(580,len(line)):              ### 종합분석일 때는 제조사와 코멘트가 별도로 들어감
                if "시험구분<" in line[k]:                ### 클레임분석일 때는 클레임날짜, 장소, 내용, 외부고객 등의 정보가 추가됨 (table 양식으로 구성)
                    if "CHECKED" in line[k+2]:
                        comments=""
                        for j in range(20):
                            if "</TD>" in line[l+40+j]:
                                break
                            else:
                                if j==0:
                                    comments=(line[l+40][22:-1])                    
                                else:
                                    comments=comments+"<br>"+line[l+40+j][:-1]
                        test.append(comments)
                        test.append(line[l+35][22:-1])                            ## 제조사 추가
                    else:
                        test.append("클레임 날짜: "+line[l+8][22:-1]+", 클레임 장소: "+line[l+13][22:-1]+", 클레임 내용: "+line[l+18][22:-1]+", 외부고객: "+line[l+23][22:-1])
        if "KeyWord<" in line[l]:
            test.append(line[l+1][32:-6])
        if "분석조건<" in line[l]:
            test.append(line[l+1][32:-6])                     ## 두번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석의견<" in line[l]:
            test.append(line[l+1][33:-6])                     ## 세번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석장비<" in line[l]:
            test.append(line[l+1][32:-6])          ### 두번째 값을 취해야 함 (702, 984)
    test.append("클레임 대응")    
    test_C=pd.DataFrame(test).T
    for k in range(580,len(line)):
        if "시험구분" in line[k]:                            
            if "CHECKED" in line[k+2]:                      ### 시험구분에서 종합분석일 경우 제조사 항목 추가
                test_C.columns=['의뢰번호','의뢰일','담당자','접수일','처리일2','제목',
'의뢰목적','Comments','제조사','의뢰자','처리일',
'STATUS','분석조건','분석의견','분석장비','KeyWord','의뢰유형']
            else:                                           ### 시험구분에서 클레임분석일 경우 제조사 항목 없음
                test_C.columns=['의뢰번호','의뢰일','담당자','접수일','처리일2','제목',
'의뢰목적','Comments','의뢰자','처리일',
'STATUS','분석조건','분석의견','분석장비','KeyWord','의뢰유형']
## 최종 취합 후에 접수일2, 분석장비 2 삭제 필요
    return test_C


## 환경유해물질분석 case (U)

In [8]:
def tas_u(line):
    test=[]
    for l in range(600,len(line)):
        if "담당자</" in line[l]:
            test.append(line[l+2][1:-1])
        if "의뢰번호</" in line[l]:
            test.append(line[l+1][20:30]) 
        if "성명</" in line[l]:
            test.append(line[l+1][32:-6])             ## 사람 이름이 세글자가 아닌경우 고려
        if "제목</" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰일</" in line[l]:
            test.append(line[l+1][20:-6])
        if "접수일</" in line[l]:
            test.append(line[l+1][20:-6])
        if "처리일</" in line[l]:
            test.append(line[l+1][32:-6])            ### 두번째 값을 취해야 함 (650, 970)
        if "STATUS</" in line[l]:
            test.append(line[l+1][32:-6])            ### 진행중인 경우 STATUS 안나옴!!!
        if "의뢰목적</" in line[l]:
            test.append(line[l+1][32:-6])
        if "의뢰유형</" in line[l]:
            test.append(line[l+8][9:-1])
        if "제조사" in line[l]:
            test.append(line[l+13][22:-6])
        if "분석 요청<" in line[l]:
            test.append("시료명: "+line[l+25][22:-6]+", Lot No: "+line[l+26][22:-6]+", 재질: "+line[l+27][22:-6]+", 제품명: "+line[l+28][22:-6]+", 시험항목: "+line[l+29][22:-6]+", 형태: "+line[l+30][22:-6]+", 원료명: "+line[l+31][22:-6]+", 색상: "+line[l+33][22:-6])
        if "KeyWord</" in line[l]:
            test.append(line[l+1][32:-6])
        if "분석조건</" in line[l]:
            test.append(line[l+2][2:-1])                     ## 두번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석의견</" in line[l]:
            test.append(line[l+2][2:-1])                     ## 세번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
        if "분석장비</" in line[l]:
            test.append(line[l+1][32:-6])          ### 두번째 값을 취해야 함 (702, 984)
#test.append(line[970][32:-6])                 ## 읽어들여야하는 행번호를 지정 시 중간에 라인 늘어나는 경우 오류 발생
    test_U=pd.DataFrame(test).T
    test_U.columns=['의뢰번호','의뢰일','담당자','접수일','처리일2','제목',
'의뢰목적','의뢰유형','Comments','제조사', '의뢰자','처리일',
'STATUS','분석조건','분석의견','분석장비','KeyWord']
## 최종 취합 후에 접수일2, 분석장비 2 삭제 필요
    return test_U

## 본 구문

In [16]:
# 시작시간
start = time.time()

# define 된 함수에서 각각 txt를 읽어낸 후 Dataframe 생성하게 되면,
# 이를 기존 DataFrame에 concat으로 붙이기 위해서 아무것도 없는 빈 행을 가진 DataFrame을 하나 만들어둠
# 맨 마지막에 이 행은 삭제해야 함

test2=['0'] 
result2=pd.DataFrame(test2)
result2.columns=['의뢰번호']

# 폴더 내 전체 txt 파일을 읽어서 읽어오기 
wrong_list=[]                                    ## File명과 실제 의뢰번호가 다른 경우에 의뢰번호 저장을 위함
ve_list=[]                                       ## ValueError 발생하는 의뢰번호 저장을 위함
count_check=0                                    ## 아래 for l을 580~650 행으로 한정하면서 오류가 발생할 가능성을 예측하기 위함
for lis in list_1:
    f=open(path1+"/"+lis,'r', encoding='CP949')  ## 여러 파일 불러오기
    line=f.readlines()    
    for l in range(580,650):                     ## for 문을 모든 라인을 다 돌리면 시간이 오래걸리므로, 범위를 좁혔음. 단, 에러 발생 여부를 확인하기 위해서 count_check 를 통해 누락된 샘플이 있는지 확인
        try:
            if "의뢰번호<" in line[l]:                ## '의뢰번호' 로 하는 경우 comment 내용 중에 '의뢰번호'가 들어갈 시 에러 발생. 이를 피하기 위해서 바로 다음 태그인 <를 추가
                count_check+=1
                req_code=line[l+1][24]
                wrong_list.append(lis) if lis!=(line[l+1][20:30]+'.txt') else 3         ## 잘못입력한 것 추리는 구문(txt 파일명과 실제 분석의뢰번호가 다른 케이스)  ## else 3 은 안넣으면 syntaxError 발생해서 그냥 넣은 것 (의미 x)
                if req_code in 'TP':            ## req_code=='T' or req_code=='P' 이거보다 간결함
                    result1=tas_t(line)    
                elif req_code in'ES':
                    result1=tas_e(line)    
                elif req_code=='I':
                    result1=tas_i(line)    
                elif req_code=='V':
                    result1=tas_i(line)    
                elif req_code=='O':
                    result1=tas_o(line)    
                elif req_code=='G':
                    result1=tas_g(line)    
                elif req_code=='C':
                    result1=tas_c(line)  
                elif req_code=='U':
                    result1=tas_u(line)  
        except ValueError:                            ## valueError 가 발생하는 것만 따로 빼내는 문구 (comment에 특정 문구가 있거나, 분석결과가 업데이트 안되어서 열의 수가 안맞는 경우에 발생하더라)
            ve_list.append(line[l+1][20:30])

    result2=pd.concat([result1,result2])
    f.close()                                          ##  txt file 닫아주기
    
# DataFrame 정리
result2.drop(['접수일2','처리일2','분석장비2','의뢰유형2'], axis=1, inplace=True)
if '제조사' not in result2.columns:                     ##(E code만 생성되는 경우 제조사가 없음)
    result2=result2[['의뢰번호', '의뢰자', '제목', '의뢰일', '접수일', '처리일', 'STATUS', '담당자', '의뢰목적', '의뢰유형',
       'Comments', 'KeyWord', '분석조건', '분석의견', '분석장비']]             ## 요청대로 순서 및 제목명 바꾸기
else:
    result2=result2[['의뢰번호', '의뢰자', '제목', '의뢰일', '접수일', '처리일', 'STATUS', '담당자', '의뢰목적', '의뢰유형', '제조사',
       'Comments', 'KeyWord', '분석조건', '분석의견', '분석장비']]             ## 요청대로 순서 및 제목명 바꾸기    
result2.rename(columns={'의뢰자':'의뢰인','제목':'의뢰제목','Comments':'Comment','KeyWord':'Key word'}, inplace=True)
result2.reset_index(drop=True, inplace=True)          ## 인덱스 리셋
result2.drop(len(result2)-1, inplace=True)            ## 마지막행 삭제

# 종료시간
end = time.time()

# 설명
print("소요시간: %d분 %0.2f초" %((end - start)//60, (end - start)%60))
print("전체 txt file 수: %d개" %len(list_1))
print("누락된 file 수: %d개" %(count_check-len(list_1)))  ## 누락 샘플 확인용, 0이 아니면 580~650 행을 벗어나는 구간에 의뢰번호가 있는 txt 파일이 있다는 의미
print("파일명과 의뢰번호가 다른 txt file 수: %d개" %len(wrong_list))  ## wrong_list 갯수
print("")


## ValueError 부결 체크
print("ValueError File 수: %d개" %len(ve_list))
print("(ValueError 대상 File은 xlsx에 등록되지 않음)")
print(ve_list)
for ve in ve_list:
    f=open(path1+"/"+ve+".txt",'r', encoding='CP949')  ## 여러 파일 불러오기
    line=f.readlines()    
    for l in range(580,len(line)):             
        if "CHECKED >부결" in line[l]:
            print(ve, "부결")
    f.close()

소요시간: 0분 0.29초
전체 txt file 수: 41개
누락된 file 수: 0개
파일명과 의뢰번호가 다른 txt file 수: 0개

ValueError File 수: 0개
(ValueError 대상 File은 xlsx에 등록되지 않음)
[]


## 엑셀파일 생성
* 파일리스트: old_tas.xlsx
* txt파일 제목과 본문의 의뢰번호가 서로다른 파일리스트: old_tas_wrong_list.xlsx

In [17]:
# save_name=input('파일명을 입력하세요: ')
#save_name="old_tas"
#writer = pd.ExcelWriter(save_name+'.xlsx', engine='xlsxwriter')

#test_T.to_excel(writer, sheet_name= 'T')
result2.to_excel(excel_writer='old_tas.xlsx', sheet_name='sheet1')
wl=pd.DataFrame(wrong_list)                                     ## list 를 DataFrame으로 변환
wl.to_excel(excel_writer='old_tas_wrong_list.xlsx', index=False, header=False)


In [13]:
# 에러 발생 시 txt 파일 닫는 구문
f.close()

In [ ]:
#result2.head()

## txt 파일 제목을 의뢰번호로 변경하여 새 폴더에 저장
    - 폴더 A의 모든 파일을 열어서 폴더 B에 의뢰번호 이름으로 동일한 txt 파일을 생성
    - 기존폴더: path1, 넷째 줄 ('A'로 표기)
    - 신규폴더: new_folder, 여섯째 줄 ('B'로 표기)
    ※ 폴더 내에 기존에 동일 파일이 있는 경우 덮어쓰기 되므로 주의

In [15]:
## ls_env 가상환경에서 실행

import os
import time                              ## 작업 시 소요시간을 체크하기 위함 (본 구문에 start, end 로 계산)
import pandas as pd

path1= "./TAS"                           ## 파일들이 담겨있는 folder 주소
list_txt = os.listdir(path1)               ## 해당 폴더의 파일목록을 list_1에 입력

new_folder="B"                    ## 이동할 폴더 명 지정
try:
    os.mkdir(new_folder)               ## 폴더가 있는 경우에는 에러발생하므로 pass 구문 추가
except:
    pass

# 시작시간
start = time.time()

for lis in list_txt:
    f1=open(path1+"/"+lis,'r', encoding='CP949') ## 의뢰번호 추출용 / encoding 을 하지 않으면 한글 깨짐
    f2=open(path1+"/"+lis,'r')                   ## txt 본문 복사용, encoding 을 붙이면 read 명령이 먹지 않음
    
    ## 의뢰번호 추출
    line=f1.readlines()
    
    for l in range(580,len(line)):
        if "의뢰번호<" in line[l]:
            req_no=line[l+1][20:30]          
    
    ## 새로 생성한 폴더에 txt파일 생성
    g=open('./'+new_folder+'/'+req_no+'.txt','w')  
    data=f2.read()
    g.write(data)
    
    f1.close()
    f2.close()
    g.close()
    

# 종료시간
end = time.time()

# 설명
print("소요시간: %d분 %0.2f초" %((end - start)//60, (end - start)%60))
print("전체 txt file 수: %d개" %len(list_txt))

소요시간: 4분 20.85초
전체 txt file 수: 3876개


## 참고

In [24]:
# txt 파일 하나 읽어서 의뢰번호와 코드(알파벳) 따오기 (년도+코드+월+일련번호)
#f=open("an030d[e].txt",'r', encoding='CP949')
f=open("./TAS/2007U02001.txt",'r', encoding='CP949')  ## 개별 파일 불러오기 
line=f.readlines()

for l in range(580,len(line)):
    if "의뢰번호<" in line[l]:
        print(line[l+1][20:30], line[l+1][24]) 
    if "CHECKED >부결" in line[l]:
        print("부결")

2007U02001 U


In [ ]:
# 폴더 내 전체 txt 파일을 읽어서 의뢰번호만 추출 (모든 txt의 동일한 위치에 의뢰번호가 있는지 확인용 - 실제 실행 시는 필요 x) 
count_a=0
count_line=[]
for lis in list_1:
    f=open(path1+"/"+lis,'r', encoding='CP949')  ## 여러 파일 불러오기
    line=f.readlines()
    for l in range(580,650):
        if "의뢰번호<" in line[l]:
            count_a+=1
            count_line.append(line[l+1][24]+str(l+1))  ## 코드 별 의뢰번호가 나오는 행번호
#            print(line[l+1][20:30], l+1) 
print (count_a-len(list_1))
print (set(count_line))

In [ ]:
## 위치를 확인하기 위한 작업 (실제 코드에는 사용되지 않음)
for l in range(len(line)):
    if "담당자" in line[l]:
        print ("담당자", l+2, line[l+2][1:-1])
    if "의뢰번호" in line[l]:
        print ("의뢰번호", l+1, line[l+1][20:30]) 
    if "성명" in line[l]:
        print ("성명", l+1, line[l+1][32:-6])             ## 사람 이름이 세글자가 아닌경우 고려
    if "제목<" in line[l]:
        print ("제목", l+1, line[l+1][32:-6])
    if "의뢰일" in line[l]:
        print ("의뢰일", l+1, line[l+1][20:-6])
    if "접수일" in line[l]:
        print ("접수일", l+1, line[l+1][20:-6])
    if "처리일" in line[l]:
        print ("처리일", l+1, line[l+1][32:-6])
    if "STATUS" in line[l]:
        print ("STATUS", l+1, line[l+1][32:-6])            ### 진행중인 경우 STATUS 안나옴!!!
    if "의뢰목적" in line[l]:
        print ("의뢰목적", l+1, line[l+1][32:-6])
    if "의뢰유형" in line[l]:
        print ("의뢰유형", l+8, line[l+8][9:-1])
    if "제조사" in line[l]:
        print ("제조사", l+13, line[l+13][22:-6])
#    if "Comments" in line[l]:
#        print ("Comments", l+1, line[l+1][32:-1])
    if "분석 요청<" in line[l]:
        print ("Comments", l+25, "시료명: ",line[l+25][22:-6], "Lot No: ",line[l+26][22:-6], "재질: ",line[l+27][22:-6], "제품명: ",line[l+28][22:-6], "시험항목: ",line[l+29][22:-6], "형태: ",line[l+30][22:-6], "원료명: ",line[l+31][22:-6], "색상: ",line[l+33][22:-6])
    if "KeyWord" in line[l]:
        print ("KeyWord", l+1, line[l+1][32:-6])
    if "분석조건" in line[l]:
        print ("분석조건", l+2, line[l+2][2:-1])                     ## 두번째 값을 취해야 함.
    if "분석의견" in line[l]:
        print ("분석의견", l+2, line[l+2][2:-1])                               ## 세번째 값을 취해야 함.
    if "분석장비" in line[l]:
        print ("분석장비", l+1, line[l+1][32:-6])

In [73]:
result2.columns

Index(['의뢰번호', '의뢰일', '담당자', '접수일', '처리일', '제목', '의뢰목적', '의뢰유형', '제조사',
       'Comments', '의뢰자', 'STATUS', '분석조건', '분석의견', '분석장비', 'KeyWord'],
      dtype='object')

In [ ]:
## 해당 내용을 리스트로 저장하기 위한 작업 (실제 코드에는 사용되지 않음)
test=[]
for l in range(580,len(line)):
    if "담당자" in line[l]:
        test.append(line[l+2][1:-1])
    if "의뢰번호" in line[l]:
        test.append(line[l+1][20:30]) 
    if "성명" in line[l]:
        test.append(line[l+1][32:-6])             ## 사람 이름이 세글자가 아닌경우 고려
    if "제목" in line[l]:
        test.append(line[l+1][32:-6])
    if "의뢰일" in line[l]:
        test.append(line[l+1][20:-6])
    if "접수일" in line[l]:
        test.append(line[l+1][20:-6])
    if "처리일" in line[l]:
        test.append(line[l+1][32:-6])            ### 두번째 값을 취해야 함 (650, 970)
    if "STATUS" in line[l]:
        test.append(line[l+1][32:-6])            ### 진행중인 경우 STATUS 안나옴!!!
    if "의뢰목적" in line[l]:
        test.append(line[l+1][32:-6])
#    if "의뢰유형" in line[l]:
#        test.append(line[l+8][9:-1])
#    if "제조사" in line[l]:
#        test.append(line[l+1][22:-1])
    if "이력<" in line[l]:                           ### 클레임분석요청임에도 종합분석/클레임분석을 구분하여 의뢰하게 되며,
        for k in range(580,len(line)):              ### 종합분석일 때는 제조사와 코멘트가 별도로 들어감
            if "시험구분" in line[k]:                ### 클레임분석일 때는 클레임날짜, 장소, 내용, 외부고객 등의 정보가 추가됨 (table 양식으로 구성)
                if "CHECKED" in line[k+2]:
                    comments=""
                    for j in range(20):
                        if "</TD>" in line[l+40+j]:
                            break
                        else:
                            if j==0:
                                comments=(line[l+40][22:-1])                    
                            else:
                                comments=comments+line[l+40+j]
                    test.append(comments)
                    test.append(line[l+35][22:-1])
                else:
                    test.append("클레임 날짜: "+line[l+8][22:-1]+", 클레임 장소: "+line[l+13][22:-1]+", 클레임 내용: "+line[l+18][22:-1]+", 외부고객: "+line[l+23][22:-1])
    if "KeyWord" in line[l]:
        test.append(line[l+1][32:-6])
    if "분석조건" in line[l]:
        test.append(line[l+1][32:-6])                     ## 두번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
    if "분석의견" in line[l]:
        test.append(line[l+1][33:-6])                     ## 세번째 값을 취해야 하나 600 행 이후로 필터링하면 문제없음
    if "분석장비" in line[l]:
        test.append(line[l+1][32:-6])          ### 두번째 값을 취해야 함 (702, 984)
test.append("클레임 대응")    
#test_C=pd.DataFrame(test).T
#for k in range(580,len(line)):
#    if "시험구분" in line[k]:                            
#        if "CHECKED" in line[k+2]:                      ### 시험구분에서 종합분석일 경우 제조사 항목 추가
#            test_C.columns=['의뢰번호','의뢰일','담당자','접수일','처리일2','제목','의뢰목적','Comments','제조사','의뢰자','처리일','STATUS','분석조건','분석의견','분석장비','KeyWord','의뢰유형']
#        else:                                           ### 시험구분에서 클레임분석일 경우 제조사 항목 없음
#            test_C.columns=['의뢰번호','의뢰일','담당자','접수일','처리일2','제목','의뢰목적','Comments','의뢰자','처리일','STATUS','분석조건','분석의견','분석장비','KeyWord','의뢰유형']
test

In [ ]:
test_G=pd.DataFrame(test).T
test_G.columns=['의뢰번호','의뢰일','담당자','접수일','처리일2','제목',
'의뢰목적','의뢰유형','분석장비2','Comments','의뢰자','처리일',
'STATUS','분석조건','분석의견','분석장비','KeyWord']
test_G
## 최종 취합 후에 접수일2, 분석장비 2 삭제 필요

In [ ]:
# 파일명과 파일 정보의 의뢰번호가 다른 것 리스트 추출
for wl in wrong_list:
    print(wl)